# Overview

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cleanlab/cleanvision/blob/main/docs/source/tutorials/tutorial.ipynb) 

In [ ]:
!pip install -U pip
!pip install git+https://github.com/cleanlab/cleanvision.git

**After you install these packages, you may need to restart your notebook runtime before running the rest of this notebook.**

## What is CleanVision?
**CleanVision** is built to automatically detects various issues in image datasets, such as images that are: (near) duplicates, blurry, over/under-exposed, etc. This data-centric AI package is designed as a quick first step for any computer vision project to find problems in your dataset, which you may want to address before applying machine learning.


|     | Issue Type      | Description                                                                                  | Issue Key        |
|-----|------------------|----------------------------------------------------------------------------------------------|------------------|
| 1   | Light            | Images that are too bright/washed out in the dataset                                         | light            |
| 2   | Dark             | Images that are irregularly dark                                                             | dark             |
| 3   | Odd Aspect Ratio | Images with an unusual aspect ratio (i.e. overly skinny/wide)                                                       | odd_aspect_ratio |
| 4   | Exact Duplicates | Images that are exact duplicates of each other                          | exact_duplicates |
| 5   | Near Duplicates  | Images that are almost visually identical to each other (e.g. same image with different filters)                                 | near_duplicates  |
| 6   | Blurry           | Images that are blurry or out of focus                                                  | blurry           |
| 7   | Grayscale        | Images that are grayscale (lacking color)                                                            | grayscale        |
| 8   | Low Information  | Images that lack much information (e.g. a completely black image with a few white dots) | low_information  |


The **Issue Key** column specifies the name for each type of issue in CleanVision code. See our examples which use these keys to detect only particular issue types and specify nondefault parameter settings to use when checking for certain issues.

This notebook uses an example dataset, that you can download using these commands.

wget - nc 'https://cleanlab-public.s3.amazonaws.com/CleanVision/image_files.zip'

unzip -q image_files.zip

In [ ]:
!wget - nc 'https://cleanlab-public.s3.amazonaws.com/CleanVision/image_files.zip'

In [ ]:
!unzip -q image_files.zip

## Examples

### 1. Using CleanVision to detect default issue types

In [ ]:
from cleanvision.imagelab import Imagelab

# Path to your dataset, you can specify your own dataset path
dataset_path = "./image_files/"

# Initialize imagelab with your dataset
imagelab = Imagelab(data_path=dataset_path)

# Visualize a few sample images from the dataset
imagelab.visualize(num_images=8)

# Find issues
imagelab.find_issues()

The `report()` method helps you quickly understand the major issues detected in the dataset. It reports the number of images in the dataset that exhibit each type of issue, and shows example images corresponding to the most severe instances of each issue.

In [ ]:
imagelab.report()

The main way to interface with your data is via the `Imagelab` class. This class can be used to understand the issues in your dataset at a high level (global overview) and low level (issues and quality scores for each image) as well as additional information about the dataset. It has three main attributes:
- `Imagelab.issue_summary`
- `Imagelab.issues`
- `Imagelab.info`

#### imagelab.issue_summary
Dataframe with global summary of all issue types detected in your dataset and the overall prevalence of each type.

In each row:\
`issue_type` - name of the issue\
`num_images` - number of images of that issue type found in the dataset

In [ ]:
imagelab.issue_summary

#### imagelab.issues

DataFrame assessing each image in your dataset, reporting which issues each image exhibits and a quality score for each type of issue.

In [ ]:
imagelab.issues.head()

There is a Boolean column for each issue type, showing whether each image exhibits that type of issue or not. For example,  the rows where the `is_dark_issue` column contains `True`, those rows correspond to images that appear too **dark**.

For the **dark** issue type (and more generally for other types of issues), there is a numeric column `dark_score`, which assesses how severe this issue is in each image. These quality scores lie between 0 and 1, where lower values indicate more severe instances of the issue (images which are darker in this example).

One use-case for `imagelab.issues` is to filter out all images exhibiting  one particular type of issue and rank them by their quality score. Here's how to get all blurry images ranked by their `blurry_score`, note lower scores indicate higher severity:

In [ ]:
blurry_images = imagelab.issues[imagelab.issues["is_blurry_issue"] == True].sort_values(by=['blurry_score'])
blurry_image_files = blurry_images.index.tolist()

Visualize the blurry images

In [ ]:
imagelab.visualize(image_files=blurry_image_files[:4])

A shorter way to accomplish the above task is to specify an issue type in `imagelab.visualize()`. This will show images ordered by the severity of this issue within them.

In [ ]:
imagelab.visualize(issue_types=['blurry'])

#### imagelab.info

This is a nested dictionary containing statistics about the images and other miscellaneous information stored while checking for issues in the dataset. Beware: this dictionary may be large and poorly organized (it is only intended for advanced users).

Possible keys in this dict are **statistics** and a key corresponding to each issue type

In [ ]:
imagelab.info.keys()

`imagelab.info['statistics']` is also a dict containing statistics calculated on images while checking for issues in the dataset.

In [ ]:
imagelab.info['statistics'].keys()

You can see **entropy** values for each image in the dataset as shown below.

In [ ]:
imagelab.info['statistics']['entropy']

#### Duplicate sets
`imagelab.info` can also be used to retrieve which images are near or exact  duplicates of each other. 

`issue.summary` shows the number of exact duplicate images but does not show how many such *sets* of duplicates images exist in the dataset. To see the number of exact duplicate sets, you can use `imagelab.info`

In [ ]:
imagelab.info['exact_duplicates']['num_sets']

You can also get exactly which images are there in each (exact/near) duplicated set using `imagelab.info`.

In [ ]:
imagelab.info['exact_duplicates']['sets']

**The rest of this notebook demonstrates more advanced/customized workflows you can do with CleanVision.**

### 2. Using CleanVision to detect specific issues

It might be the case that only a few issue types are relevant for your dataset and you don't want to run it through all checks to save time. You can do so by specifying `issue_types` as an argument.

`issue_types` is a dict, where keys are the issue types that you want to detect and values are dict which contains hyperparameters. This example uses default hyperparameters, in which case you can leave the hyperparameter dict empty. To find keys for issue types check the above table that lists all issue types supported by CleanVision. 

In [ ]:
# Initialize imagelab with your dataset
imagelab = Imagelab(data_path=dataset_path)

# specify issue types to detect
issue_types = {"dark": {}}

# Find issues
imagelab.find_issues(issue_types)

# Show a report of the issues found
imagelab.report()

### 3. Check for additional types of issues using the same instance

Suppose you also want to check for blurry images  after having already detected dark images in the dataset. You can use the **same** Imagelab instance to incrementally check for another type of issue like  blurry images.

In [ ]:
issue_types = {"blurry": {}}

imagelab.find_issues(issue_types)

imagelab.report()

### 4. Save and load

CleanVision also has a save and load functionality that you can use to save the results and load them at a later point in time to see results or run more checks.

For saving, specify `force=True` to overwrite existing files.

In [ ]:
save_path = "./results"
imagelab.save(save_path)

For loading a saved instance, specify `dataset_path` to help check for any inconsistencies between dataset paths in the previous and current run.

In [ ]:
imagelab = Imagelab.load(save_path, dataset_path)

### 5. Check for an issue with a different threshold

You can use the loaded imagelab instance to check for an issue type with a custom hyperparameter. Here is a table of hyperparameters that each issue type supports and their permissible values. 

`threshold`- All images with scores below this threshold will be flagged as an issue.

`hash_size` - This controls how much detail about an image we want to keep for getting perceptual hash. Higher sizes imply more detail.

`hash_type` - Type of perceptual hash to use. Currently `whash` and `phash` are the supported hash types. Check [here](https://github.com/JohannesBuchner/imagehash) for more details on these hash types.

|   | Issue Key        | Hyperparameters                                   |
|---|------------------|---------------------------------------------------|
| 1 | light            | threshold (between 0 and 1)                       |
| 2 | dark             | threshold (between 0 and 1)                       |
| 3 | odd_aspect_ratio | threshold (between 0 and 1)                       |
| 4 | exact_duplicates | N/A                                               |
| 5 | near_duplicates  | hash_size (power of 2), hash_types (whash, phash) |
| 6 | blurry           | threshold (between 0 and 1)                       |
| 7 | grayscale        | threshold (between 0 and 1)                       |
| 8 | low_information  | threshold (between 0 and 1)                       |

In [ ]:
issue_types = {"dark": {"threshold": 0.2}}
imagelab.find_issues(issue_types)

imagelab.report()

Note the number of images with dark issue has reduced from the previous run.

### 6. Run CleanVision for default issue types, but override hyperparameters for one or more issues

In [ ]:
imagelab = Imagelab(data_path=dataset_path)

# Check for all default issue types
imagelab.find_issues()

# Specify an issue with custom hyperparameters
issue_types = {"odd_aspect_ratio": {"threshold": 0.2}}

# Run find issues again with specified issue types
imagelab.find_issues(issue_types)


# Pass list of issue_types to imagelab.report() to report only those issue_types
imagelab.report(["odd_aspect_ratio", "low_information"])

### 7. Customize report

Report can also be customized in various ways to help with the analysis. For example, you can change the verbosity to return more or less information on issues found, default is `verbosity=1`

In [ ]:
# Change verbosity
imagelab.report(verbosity=2)

You may want to exclude issues from your report which are prevalent in say more than 50% of the dataset and are not real issues but just how the dataset is, for example dark images in an astronomy dataset may not be an issue. You can use the `max_prevalence` parameter in report to exclude such issues. In this example all issues present in more than 3% of the dataset are excluded.

In [ ]:
imagelab.report(max_prevalence=0.03)

### 8. Visualize specific issues

Imagelab provides `imagelab.visualize` that you can use to see examples of specific issues in your dataset.

`num_images` and `cell_size` are optional arguments, that you can use to control number of examples of each issue type and size of each image in the grid respectively.

In [ ]:
issue_types = ["grayscale"]
imagelab.visualize(issue_types=issue_types, num_images=8, cell_size=(3, 3))

## Advanced: Create your own issue type

You can also create a custom issue type by extending the base class `IssueManager`. CleanVision can then detect your custom issue along with other pre-defined issues in any image dataset! Here's an example of a custom issue manager, which can also be found in the [examples/](https://github.com/cleanlab/cleanvision/blob/main/examples/custom_issue_manager.py) folder of the source code.

In [ ]:
from typing import Any, Dict, List, Optional

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

from cleanvision.dataset.base_dataset import Dataset
from cleanvision.issue_managers import register_issue_manager
from cleanvision.utils.base_issue_manager import IssueManager
from cleanvision.utils.utils import get_is_issue_colname, get_score_colname

ISSUE_NAME = "custom"


@register_issue_manager(ISSUE_NAME)
class CustomIssueManager(IssueManager):
    """
    Example class showing how you can self-define a custom type of issue that
    CleanVision can simultaneously check your data for alongside its built-in issue types.
    """

    issue_name: str = ISSUE_NAME
    visualization: str = "individual_images"

    def __init__(self) -> None:
        super().__init__()
        self.params = self.get_default_params()

    def get_default_params(self) -> Dict[str, Any]:
        return {"threshold": 0.4}

    def update_params(self, params: Dict[str, Any]) -> None:
        self.params = self.get_default_params()
        non_none_params = {k: v for k, v in params.items() if v is not None}
        self.params = {**self.params, **non_none_params}

    @staticmethod
    def calculate_mean_pixel_value(image: Image.Image) -> float:
        gray_image = image.convert("L")
        return np.mean(np.array(gray_image))

    def get_scores(self, raw_scores: List[float]) -> "np.ndarray[Any, Any]":
        scores = np.array(raw_scores)
        return scores / 255.0

    def mark_issue(self, scores: pd.Series, threshold: float) -> pd.Series:
        return scores < threshold

    def update_summary(self, summary_dict: Dict[str, Any]) -> None:
        self.summary = pd.DataFrame({"issue_type": [self.issue_name]})
        for column_name, value in summary_dict.items():
            self.summary[column_name] = [value]

    def find_issues(
        self,
        *,
        params: Optional[Dict[str, Any]] = None,
        dataset: Optional[Dataset] = None,
        imagelab_info: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        super().find_issues(**kwargs)
        assert params is not None
        assert imagelab_info is not None
        assert dataset is not None

        self.update_params(params)

        raw_scores = []
        for idx in tqdm(dataset.index):
            image = dataset[idx]
            raw_scores.append(self.calculate_mean_pixel_value(image))

        score_colname = get_score_colname(self.issue_name)
        is_issue_colname = get_is_issue_colname(self.issue_name)

        scores = pd.DataFrame(index=dataset.index)
        scores[score_colname] = self.get_scores(raw_scores)

        is_issue = pd.DataFrame(index=dataset.index)
        is_issue[is_issue_colname] = self.mark_issue(
            scores[score_colname], self.params["threshold"]
        )

        self.issues = pd.DataFrame(index=dataset.index)
        self.issues = self.issues.join(scores)
        self.issues = self.issues.join(is_issue)

        self.info[self.issue_name] = {"PixelValue": raw_scores}
        summary_dict = self._compute_summary(
            self.issues[get_is_issue_colname(self.issue_name)]
        )

        self.update_summary(summary_dict)

### 9. Run CleanVision with a custom issue

In [ ]:
imagelab = Imagelab(data_path=dataset_path)

issue_name = CustomIssueManager.issue_name


# To ensure your issue manager is registered, check list of possible issue types
# issue_name should be present in this list
imagelab.list_possible_issue_types()

In [ ]:
issue_types = {issue_name: {}}
imagelab.find_issues(issue_types)
imagelab.report()

**Beyond the collection of image files demonstrated here, you can alternatively run CleanVision on: [Hugging Face datasets](https://github.com/cleanlab/cleanvision/blob/main/docs/source/tutorials/huggingface_dataset.ipynb) and [torchvision datasets](https://github.com/cleanlab/cleanvision/blob/main/docs/source/tutorials/torchvision_dataset.ipynb).**